In [ ]:
!nvidia-smi

## Import

In [ ]:
from define_func import *
from train_model import *
from inference_model import *
from modeling.deeplab import DeepLab

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
torch.cuda.is_available()

In [ ]:
root_directory = "./"
search_keyword = "__pycache__"
delete_folders_with_keyword(root_directory, search_keyword)

## Data Loader

In [ ]:
dataset = CustomDataset(csv_file='./train_source.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=9, shuffle=True, num_workers=4)

target_dataset = CustomDataset(csv_file='./train_target.csv', transform=transform, infer=True)
target_dataloader = DataLoader(target_dataset, batch_size=12, shuffle=True, num_workers=4)

print("dataset length: ", len(dataset))
print("target_dataset length: ", len(target_dataset))

In [ ]:
#weight = class_wieghts(dataset)

#weight = torch.tensor(weight).to(device)

#weight = torch.tensor([ 0.0700,  1.3700,  0.0600,  1.0300,  1.6400,  3.5700,  1.4100,  0.0800, 0.0500,  7.1400, 10.0000,  0.1200,  0.0400]).to(device)

#print(weight)

# 0: road, 1: sidewalk, 2: construction, 3: fence, 4: pole, 5: traffic light, 6: traffic sign, 7: nature, 8: sky, 9: person, 10: rider, 11: car, 12: background

"""
각 원소 개수: [26252822, 1369290, 33282797, 1823176, 1150106, 526593, 1345478, 24846281, 39749060, 256053, 182491, 15785355, 42149602]
각 원소의 비율(소수점 둘째 자리까지): [13.91, 0.73, 17.64, 0.97, 0.61, 0.28, 0.71, 13.17, 21.06, 0.14, 0.1, 8.36, 22.33]
각 원소의 가중치(소수점 둘째 자리까지): [0.07, 1.37, 0.06, 1.03, 1.64, 3.57, 1.41, 0.08, 0.05, 7.14, 10.0, 0.12, 0.04]
각 원소의 가중치(총합 1): [0.0026335590669676447, 0.05154251316779534, 0.002257336343115124, 0.03875094055680963, 0.061700526711813385, 0.13431151241534986, 0.05304740406320541, 0.0030097817908201654, 0.0018811136192626034, 0.2686230248306997, 0.37622272385252065, 0.004514672686230248, 0.0015048908954100827]
0.9999999999999999
tensor([ 0.0700,  1.3700,  0.0600,  1.0300,  1.6400,  3.5700,  1.4100,  0.0800,
         0.0500,  7.1400, 10.0000,  0.1200,  0.0400])
"""

## Define Model

In [ ]:
# DeepLabV3+
model = DeepLab(backbone='drn', output_stride=16, num_classes=13).to(device)
# loss function과 optimizer 정의
#criterion = torch.nn.CrossEntropyLoss()
#criterion = monai.losses.DiceLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=weight)
#criterion = monai.losses.GeneralizedDiceFocalLoss(to_onehot_y=True, softmax=True, weight=None, lambda_gdl=0.3, lambda_focal=0.7)
criterion = monai.losses.DiceFocalLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=None, lambda_dice=1.0, lambda_focal=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 200, 250], gamma=1)
epochs = 100
model_name = "DeepLabV3+"
backbone_name = "drn"

## Model Train

In [ ]:
TrainModel(device=device, dataloader=dataloader, target_dataloader=target_dataloader, model=model,
           criterion=criterion, optimizer=optimizer, scheduler=scheduler, epochs=epochs,
           model_name=model_name, backbone_name=backbone_name)

In [ ]:
TrainModel(device=device, dataset=dataset, dataloader=dataloader, model=model,
           criterion=criterion, optimizer=optimizer, scheduler=scheduler, epochs=epochs,
           model_name=model_name, backbone_name=backbone_name)

## Resume Model Train

In [ ]:
"""# DeepLabV3+
# model 초기화
model = DeepLab(backbone='drn', output_stride=16, num_classes=13).to(device)
# loss function과 optimizer 정의
#criterion = torch.nn.CrossEntropyLoss()
#criterion = monai.losses.DiceLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=weight)
criterion = monai.losses.DiceFocalLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=None, lambda_dice=1.0, lambda_focal=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[150, 200, 250], gamma=1)
epochs = 100
model_name = "DeepLabV3+"
backbone_name = "drn"

checkpoint = torch.load('./model/DeepLabV3+_drn_checkpoint-0080.pth')
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
last_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]

print(checkpoint_description)"""

In [ ]:
"""TrainModel(device=device, dataloader=dataloader, target_dataloader=target_dataloader, model=model,
           criterion=criterion, optimizer=optimizer, scheduler=scheduler, epochs=epochs,
           model_name=model_name, backbone_name=backbone_name, last_epoch=last_epoch)"""

## mIoU

In [ ]:
val_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform, infer=True)
val_dataloader = DataLoader(val_dataset, batch_size=9, shuffle=False, num_workers=4)

print(len(val_dataset))

In [ ]:
# DeepLabV3+
# model 초기화
model = DeepLab(backbone='drn', output_stride=16, num_classes=13).to(device)
# loss function과 optimizer 정의
#criterion = torch.nn.CrossEntropyLoss()
#criterion = monai.losses.DiceLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=weight)
criterion = monai.losses.DiceFocalLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=None, lambda_dice=1.0, lambda_focal=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model_name = "DeepLabV3+"
backbone_name = "drn"

checkpoint = torch.load(f'./model/{model_name}_{backbone_name}_checkpoint-0100.pth')
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
last_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]

print(checkpoint_description)

In [ ]:
InferModel_val(device=device, dataset=val_dataset, dataloader=val_dataloader, model=model,
               criterion=criterion, optimizer=optimizer, last_epoch=last_epoch,
               model_name=model_name, backbone_name=backbone_name)

## Inference

In [ ]:
test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=9, shuffle=False, num_workers=4)

print(len(test_dataset))

In [ ]:
# DeepLabV3+
# model 초기화
model = DeepLab(backbone='drn', output_stride=16, num_classes=13).to(device)
# loss function과 optimizer 정의
#criterion = torch.nn.CrossEntropyLoss()
#criterion = monai.losses.DiceLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=weight)
criterion = monai.losses.DiceFocalLoss(to_onehot_y=True, softmax=True, jaccard=True, weight=None, lambda_dice=1.0, lambda_focal=1.0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
model_name = "DeepLabV3+"
backbone_name = "drn"

checkpoint = torch.load(f'./model/{model_name}_{backbone_name}_checkpoint-0100.pth')
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
last_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]

print(checkpoint_description)

In [ ]:
InferModel(device=device, dataloader=test_dataloader, model=model,
           criterion=criterion, optimizer=optimizer, last_epoch=last_epoch,
           model_name=model_name, backbone_name=backbone_name)

In [ ]:
InferModel(device=device, dataset = test_dataset, dataloader=test_dataloader, model=model,
           criterion=criterion, optimizer=optimizer, last_epoch=last_epoch,
           model_name=model_name, backbone_name=backbone_name)